# Import Libraries

In [1]:
import datetime,GetOldTweets3,IPython,pandas,sqlite3,time,urllib

### Creates a sqlite database to store tweets downloaded
sqlite3 is included in Python's packages. It can create an SQL database that runs locally without a server. The benefit of using sqlite is that the SQL commands are not saved in the database until 'commit()' function is called. This is useful when working with downloads since sometimes the connection can be lost in the middle of the script and cause corruption to the data. Using 'commit()' in sqlite only after the downloads is complete safeguards against such data corruption incidents. The date last downloaded is also saved in such a database so that the download job can be interrupted and continued with ease.
>Previous runs have encountered this issue of tweets producing an error during GetOldTweets3 tweet-cleaning function.
>https://github.com/Jefferson-Henrique/GetOldTweets-python/issues/163
>The solution is to download the tweet as-is and process the cleaning afterwards.
>The following edit was done to bypass the tweet-cleaning function:
>Edit from line 87 of '\site-packages\GetOldTweets3\manager\TweetManager.py'
><br>
><br>tweet.to = usernames[1] if len(usernames) >= 2 else None  # take the first recipient if many
><br>Comment out this --> rawtext = TweetManager.textify(tweetPQ("p.js-tweet-text").html(), tweetCriteria.emoji)
><br>Comment out this --> tweet.text = re.sub(r"\s+", " ", rawtext).replace('# ', '#').replace('@ ', '@').replace('$ ', '$')
><br>Add this --> tweet.text = tweetPQ("p.js-tweet-text").html()

### Creates the [date_range] table with the date we intend to start.

In [2]:
# This cell should not be run if we wish to continue from where we left off (e.g. connection is lost midway during download).
conn01 = sqlite3.connect('twitter_us.db')
cur_sqlite = conn01.cursor()

cur_sqlite.execute('CREATE TABLE IF NOT EXISTS [date_range] ([since],[until]);')

cur_sqlite.execute('''
DELETE FROM [date_range];''')

cur_sqlite.execute('''
INSERT INTO [date_range]
VALUES("2020-04-01","2020-04-02");''')

conn01.commit()
conn01.close()

### Start downloading

In [3]:
# create/connect to twitter_us.db. The file will be created if it doesn't exist.
conn01 = sqlite3.connect('twitter_us.db',
                         detect_types = sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES) # this line is for storing dates in sqlite
cur_sqlite = conn01.cursor()

In [4]:
time_start = datetime.datetime.now()
tweet_volume = 0
tweet_batch = 1800
tweet_states = ''
run_days = 3650

tweet_attr = ['id',
              'permalink',
              'username',
              'to',
              'text',
              'date',
              'replies',
              'retweets',
              'favorites',
              'mentions',
              'hashtags',
              'geo']

us_states = ['Alabama',
             'Alaska',
             'Arizona',
             'Arkansas',
             'California',
             'Colorado',
             'Connecticut',
             'Delaware',
             'Florida',
             'Georgia',
             'Hawaii',
             'Idaho',
             'Illinois',
             'Indiana',
             'Iowa',
             'Kansas',
             'Kentucky',
             'Louisiana',
             'Maine',
             'Maryland',
             'Massachusetts',
             'Michigan',
             'Minnesota',
             'Mississippi',
             'Missouri',
             'Montana',
             'Nebraska',
             'Nevada',
             'New Hampshire',
             'New Jersey',
             'New Mexico',
             'New York',
             'North Carolina',
             'North Dakota',
             'Ohio',
             'Oklahoma',
             'Oregon',
             'Pennsylvania',
             'Rhode Island',
             'South Carolina',
             'South Dakota',
             'Tennessee',
             'Texas',
             'Utah',
             'Vermont',
             'Virginia',
             'Washington',
             'West Virginia',
             'Wisconsin',
             'Wyoming']

SQL_str = ''

for i in tweet_attr:
    SQL_str += '['+i+']'
    if i == tweet_attr[-1]:
        pass
    elif i == 'date':
        SQL_str += ' timestamp,'
    else:
        SQL_str += ','

# Create a [twitter] table where the tweets will be saved in.
cur_sqlite.execute('''
CREATE TABLE IF NOT EXISTS [twitter] (
[US],
'''+SQL_str+''');''')

# Create a temporary table for the tweets to be stored before being saved into the actual [twitter] table.
cur_sqlite.execute('''
CREATE TABLE IF NOT EXISTS [temp] (
[US],
'''+SQL_str+''');''')

# This function inserts the tweets into the [temp] table as the tweet is downloaded one tweet at a time.
# This function can be called from the GetOldTweets3 downloading function.
def tweet_buffer(tweets):
    global time_start
    global tweet_volume
    global tweet_states
    tweet_volume += len(tweets)

    for i in tweets:
        cur_sqlite.execute('''
INSERT INTO [temp]
    VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?);''',
                           (tweet_states,
                            i.id,
                            i.permalink,
                            i.username,
                            i.to,
                            i.text,
                            i.date,
                            i.replies,
                            i.retweets,
                            i.favorites,
                            i.mentions,
                            i.hashtags,
                            i.geo))

# When the tweet volume reaches a certain number, twitter website will start to restrict the connection from downloading more tweets for a while.
# To prevent that, the script needs to wait for a while after downloading several scripts.
# This wait time will be shorter than if the twitter webite restricts the connection.
# While waiting, it is a good time to save the data into [twitter] table.
    if tweet_volume+tweet_batch >= 14000 or len(tweets) < tweet_batch:
        cur_sqlite.execute('''
DELETE FROM [twitter]
WHERE
    [id] IN (
SELECT
    [id]
FROM [temp]);''')

        cur_sqlite.execute('''
INSERT INTO [twitter]
SELECT
    *
FROM [temp];''')

        cur_sqlite.execute('''
DELETE FROM [temp];''')

        time_lapse = (datetime.datetime.now()-time_start).seconds
        if time_lapse < tweet_volume/12.:
            time.sleep((tweet_volume/12.)-time_lapse)
        else:
            pass
        print(str(tweet_volume)+'--'+str(time_lapse)+'-->'+str((datetime.datetime.now()-time_start).seconds))
        time_start = datetime.datetime.now()
        tweet_volume = 0
    else:
        time_lapse = (datetime.datetime.now()-time_start).seconds
        print(str(tweet_volume)+'--'+str(time_lapse))

# This is where the data is actually saved into the database. Previously all SQL commands will be 'saved' in a temporary file until this step.
    conn01.commit()

# This is the actual loop that calls the GetOldTweet3 functions to download the tweets.
# Test connection is active. If it is, run the script. Otherwise, wait for 1 minute then try again.
for i in range(0,
               run_days):
    try:
        urllib.request.urlopen(r'http://www.google.com')
        time_start = datetime.datetime.now()
        tweet_volume = 0
        tweet_batch = 1800
        tweet_states = ''

        IPython.display.clear_output()

# This is where we read the last downloaded date from the [date_range] table. We will continue the download from this date.
        df = pandas.read_sql('''
SELECT
    *
FROM [date_range];''',
                             con = conn01)

        print(df['since'][0]+' Start-> '+datetime.datetime.now().strftime('%H:%M'))

# Clear the [temp] table. This is for incidences where we have to continue after the connection was lost and we have to start again.
        cur_sqlite.execute('''
DELETE FROM [temp];''')

# For every US state, search for and download the tweets using the keywords specified in 'setQuerySearch()'.
        for j in us_states:
            tweet_states = j
            print(tweet_states)
            criteria = GetOldTweets3.manager.TweetCriteria().setQuerySearch('global warming')\
                                                            .setSince(df['since'][0])\
                                                            .setUntil(df['until'][0])\
                                                            .setNear(tweet_states)

# The TweetManager is where the action is. Tweets are getting parsed and downloaded here.
            GetOldTweets3.manager.TweetManager.getTweets(tweetCriteria = criteria,
                                                         receiveBuffer = tweet_buffer,
                                                         bufferLength = tweet_batch)

            time.sleep(1)
        print(df['since'][0]+' Done-> '+datetime.datetime.now().strftime('%H:%M'))

# Subtract 1 day to the date in [date_range] table. We have downloaded 1 day of tweet. 'commit()' is called after this.
# We move from most recent tweets to older tweets.
        df = pandas.read_sql('''
SELECT
    STRFTIME("%Y-%m-%d",
             [since],
             "-1 day") AS [since],
    STRFTIME("%Y-%m-%d",
             [until],
             "-1 day") AS [until]
FROM [date_range];''',
                             con = conn01)

        cur_sqlite.execute('''
DELETE FROM [date_range];''')

        cur_sqlite.execute('''
INSERT INTO [date_range]
    VALUES(?,?);''',
                           (df['since'][0],
                            df['until'][0]))
        conn01.commit()
    except:
        time.sleep(60)
        continue



2011-01-01 Start-> 01:03
Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
1--32-->32
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming
2011-01-01 Done-> 01:05


# Export to csv.

In [5]:
df = pandas.read_sql('''
SELECT
    [US],
    [id],
    [permalink],
    [username],
    [to],
    [text],
    STRFTIME("%Y-%m-%d %H:%M:%S",[date]) AS [date],
    [replies],
    [retweets],
    [favorites],
    [mentions],
    [hashtags],
    [geo]
FROM [twitter]''',
                     con = conn01)

df['date'] = pandas.to_datetime(df['date'])

df.to_csv(r'twitter_us.csv',
          index = False)

In [6]:
conn01.commit()
conn01.close()